<a href="https://colab.research.google.com/github/PranavkrishnaVadhyar/basics-of-genai/blob/main/RAG_GDGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Page 1: **Introduction to Generative AI**
- **What is Generative AI?**
  - Generative AI refers to models capable of creating new content such as text, images, audio, or code.
  - It uses patterns from existing data to generate realistic outputs.
- **Examples:**
  - Text generation: ChatGPT, Bard
  - Image generation: DALL-E, Stable Diffusion
  - Audio generation: DeepVoice
- **Key Applications:**
  - Content creation, personalized learning, gaming, healthcare

---

### Page 2: **How Generative AI Works**
- **Key Steps:**
  1. **Pre-training:** Models are trained on large datasets to learn patterns.
  2. **Fine-tuning:** Adapting the model to specific tasks or domains.
  3. **Generation:** Producing outputs based on prompts or inputs.
- **Techniques:**
  - Deep Learning (e.g., Transformers, GANs, VAEs)
  - Reinforcement Learning (e.g., RLHF for LLMs)

---

### Page 3: **What are Large Language Models (LLMs)?**
- **Definition:**
  - LLMs are AI models designed to understand and generate human-like text.
  - Built using transformer architectures.
- **Examples:**
  - OpenAI GPT series, Google BERT, Meta LLaMA
- **Key Features:**
  - Language understanding, text completion, summarization

---

### Page 5: **Key Terminologies**
- **Token:** Smallest unit of text (e.g., words, subwords).
- **Context Window:** Amount of text the model can "remember."
- **Embedding:** Numerical representation of text.
- **Attention Mechanism:** Focuses on relevant parts of the input.
- **Inference:** Generating outputs from a trained model.




#RAG

![Google's logo](https://miro.medium.com/v2/resize:fit:720/format:webp/0*PPVg5hRyTf34fJ5i.png)

In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install --quiet --upgrade langchain-ollama langchain-groq
!pip install --upgrade --quiet  sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.3 MB/s eta 0:00:00


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter


#RAG-ന് LLM വേണ്ടേ???🤔🤔

In [ ]:
from google.colab import userdata
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192", api_key=userdata.get("GROQ"))

#Data വിട്ടു ഒരു കളി ഇല്ല😌😌

In [ ]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


#Chunk-ing ❤️❤️

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

#Vector store: Croma📺📺 അല്ല Chroma

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-6-756cce1f5e7d>:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = Chroma.from_documents(splits, hf)

#Retriver : ഇവിടെ വാ നീ 🫴

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = db.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#Generator

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is self reflection in agents?")

'Self-reflection in agents refers to the ability to reflect on past actions, learn from mistakes, and refine future plans to improve the quality of results. This is achieved by showing the agent examples of failed trajectories and ideal reflections, which are then added to its working memory to guide future changes in the plan. Self-reflection allows agents to improve iteratively by refining past action decisions and correcting previous mistakes.'